In [1]:
# Import modules 
import pandas as pd
import numpy as np
import datetime
import time

Read in the data web scraped from spotrac.com
The data frame consists of contract information of MLB free agents ranging from 2012 to 2020. 

In [2]:
FA_list = pd.read_csv("FA_list.csv")

In [3]:
Names = list(FA_list["Name"])

In [4]:
Names[0]

'Albert Pujols'

In [5]:
FA_list.head()

,Name,Position,Age,From,To,Years,Dollars,Average_Salary,FA_year
0,Albert Pujols,DH,32,STL,LAA,10,"$240,000,000","$24,000,000",2012
1,Prince Fielder,DH,28,MIL,DET,9,"$214,000,000","$23,777,778",2012
2,Jose Reyes,SS,29,NYM,MIA,6,"$106,000,000","$17,666,667",2012
3,C.J. Wilson,SP,31,TEX,LAA,5,"$77,500,000","$15,500,000",2012
4,Mark Buehrle,SP,33,CHW,MIA,4,"$58,000,000","$14,500,000",2012


In [6]:
FA_list.shape

(1118, 9)

FA_list has 1,118 rows

## Join bbrefID from baseballdatabank
Adding baseball reference ID from basebll databank to the free agents so we could further web scrape player statistics from baseballreference.com   <br/>
__[Baseball databank](http://www.seanlahman.com/baseball-archive/statistics/)__ is a compilation of historical baseball data in a convenient, tidy format, distributed under Open Data terms.

bbrefID takes the first two letters of a players first name + the first five letters of a player's last name + two number to form. <br/>
For example: bbrefID for Albert Pujos would be pujolal01.

In [7]:
# read in data from baseball databank
People = pd.read_csv("C:\\Users\\iamia\\Desktop\\MLB Free Agent\\baseballdatabank-master\\core\\People.csv")

In [8]:
People.head(10)

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,...,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
0,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
1,aaronha01,1934.0,2.0,5.0,USA,AL,Mobile,NaN,NaN,NaN,...,Aaron,Henry Louis,180.0,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,1939.0,8.0,5.0,USA,AL,Mobile,1984.0,8.0,16.0,...,Aaron,Tommie Lee,190.0,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
3,aasedo01,1954.0,9.0,8.0,USA,CA,Orange,NaN,NaN,NaN,...,Aase,Donald William,190.0,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01
4,abadan01,1972.0,8.0,25.0,USA,FL,Palm Beach,NaN,NaN,NaN,...,Abad,Fausto Andres,184.0,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01
5,abadfe01,1985.0,12.0,17.0,D.R.,La Romana,La Romana,NaN,NaN,NaN,...,Abad,Fernando Antonio,220.0,73.0,L,L,2010-07-28,2019-09-28,abadf001,abadfe01
6,abadijo01,1850.0,11.0,4.0,USA,PA,Philadelphia,1905.0,5.0,17.0,...,Abadie,John W.,192.0,72.0,R,R,1875-04-26,1875-06-10,abadj101,abadijo01
7,abbated01,1877.0,4.0,15.0,USA,PA,Latrobe,1957.0,1.0,6.0,...,Abbaticchio,Edward James,170.0,71.0,R,R,1897-09-04,1910-09-15,abbae101,abbated01
8,abbeybe01,1869.0,11.0,11.0,USA,VT,Essex,1962.0,6.0,11.0,...,Abbey,Bert Wood,175.0,71.0,R,R,1892-06-14,1896-09-23,abbeb101,abbeybe01
9,abbeych01,1866.0,10.0,14.0,USA,NE,Falls City,1926.0,4.0,27.0,...,Abbey,Charles S.,169.0,68.0,L,L,1893-08-16,1897-08-19,abbec101,abbeych01


In [9]:
# Combine first name and last name
People["Name"] = People["nameFirst"].str.replace(" ", "") + " " + People["nameLast"]

The reason behind deleting spaces in the first name is to make sure that the initials are spaceless.    <br/>
For example, "C.J." and "C. J." will be different if we don't strip the spaces.

In [10]:
# Keep only Players born between 1970 and 1995
# Keep only Name, bbrefID to left join into FA_list
People_join = People[People.birthYear.isin(range(1970,1995))]
People_join = People_join[["Name", "bbrefID", "finalGame", "birthYear"]]
# Left join so that FA_list does not change
FA_list_joined = FA_list.merge(People_join, on = "Name", how = "left", indicator = True)

Set the range of players birth year so we limit the chances to get duplicate names and ID pairs.
Use finalGame and birthYear to further filter out players with same names. 

In [11]:
FA_list_joined.shape

(1147, 13)

In [12]:
FA_list.shape

(1118, 9)

There are 29 more rows in the joined dataframe than the original dataframe.

## Two problems:
1. Players without bbrefID (bbrefID = NaN) <br/> 
    - Some players have changed names while others have use different names to register.   <br/>
    - For example: B.J. Upton and Melvin Upton are the same player  <br/>
2. Players with multiple bbrefID (players with same names, hence some players have two bbrefID)  <br/>
    - This is becuase players have similar initials or the same names  <br/>
    - For example: There are two "Chirs Young". One is a pitcher while one in a outfielder. 

In [14]:
# Dataframe with players without bbrefID because the names are different in spotrac.com and baseball databank.
FA_list_joined[FA_list_joined["_merge"] == "left_only"]

,Name,Position,Age,From,To,Years,Dollars,Average_Salary,FA_year,bbrefID,finalGame,birthYear,_merge
27,Jerry Hairston Jr.,LF,36,MIL,LAD,2,"$6,000,000","$3,000,000",2012,NaN,NaN,NaN,left_only
57,Vincent Padilla,RP,34,LAD,BOS,1,"$1,500,000","$1,500,000",2012,NaN,NaN,NaN,left_only
91,Mark Derosa,3B,37,SF,WSH,1,"$800,000","$800,000",2012,NaN,NaN,NaN,left_only
110,Melvin Upton,LF,28,TB,ATL,5,"$72,500,000","$14,500,000",2013,NaN,NaN,NaN,left_only
143,Mariano Rivera,RP,43,NYY,NYY,1,"$10,000,000","$10,000,000",2013,NaN,NaN,NaN,left_only
238,Mark Derosa,3B,38,WSH,TOR,1,"$750,000","$750,000",2013,NaN,NaN,NaN,left_only
249,Juan Carlos Oviedo,RP,31,MIA,TB,1,"$490,000","$490,000",2013,NaN,NaN,NaN,left_only
351,Juan Carlos Oviedo,RP,32,TB,TB,1,"$1,500,000","$1,500,000",2014,NaN,NaN,NaN,left_only
379,Yorbit Torrealba,C,35,COL,LAA,1,"$950,000","$950,000",2014,NaN,NaN,NaN,left_only
387,J.C. Gutiérrez,RP,30,LAA,SF,1,"$850,000","$850,000",2014,NaN,NaN,NaN,left_only


In [15]:
# Dataframe of players with multiple bbrefID
FA_list_joined[FA_list_joined[["Name", "FA_year"]].duplicated(keep=False)]

,Name,Position,Age,From,To,Years,Dollars,Average_Salary,FA_year,bbrefID,finalGame,birthYear,_merge
2,Jose Reyes,SS,29,NYM,MIA,6,"$106,000,000","$17,666,667",2012,reyesjo01,2018-09-30,1983.0,both
3,Jose Reyes,SS,29,NYM,MIA,6,"$106,000,000","$17,666,667",2012,reyesjo02,2006-09-30,1983.0,both
32,Alex Gonzalez,SS,35,ATL,MIL,1,"$4,250,000","$4,250,000",2012,gonzaal01,2006-05-20,1973.0,both
33,Alex Gonzalez,SS,35,ATL,MIL,1,"$4,250,000","$4,250,000",2012,gonzaal02,2014-04-17,1977.0,both
34,Freddy Garcia,SP,35,NYY,NYY,1,"$4,000,000","$4,000,000",2012,garcifr01,1999-10-03,1972.0,both
35,Freddy Garcia,SP,35,NYY,NYY,1,"$4,000,000","$4,000,000",2012,garcifr03,2013-09-24,1976.0,both
80,Chris Young,RP,33,NYM,NYM,1,"$900,000","$900,000",2012,youngch03,2017-06-17,1979.0,both
81,Chris Young,RP,33,NYM,NYM,1,"$900,000","$900,000",2012,youngch04,2018-07-03,1983.0,both
159,Scott Baker,SP,31,MIN,CHC,1,"$5,500,000","$5,500,000",2013,bakersc01,1995-07-17,1970.0,both
160,Scott Baker,SP,31,MIN,CHC,1,"$5,500,000","$5,500,000",2013,bakersc02,2015-05-02,1981.0,both


##  Players without bbrefID

There is no smart way but to fill in the IDs manually. 

In [16]:
# The index of NaN bbrefID in FA_list_joined
bbrefID_NA_index = FA_list_joined[FA_list_joined["bbrefID"].isnull()].index

In [17]:
# The names of NaN bbrefID in FA_list_joined
Name_NaNID = FA_list_joined[FA_list_joined["bbrefID"].isnull()].Name

In [18]:
# Fill in the NaN manually. 
NaNID = ["hairsje02", "padilvi01", "derosma01", "uptonbj01", "riverma01", "derosma01", "nunezle01", "nunezle01", "torreyo01", "gutieju01", "maybejo02", "younger03", "masseni01", "aokino01", "pearcst01", "albural01", "joycema01", "younger03", "dunnmi01","pearcst01","joycema01", "milonto01", "sabatc.01", "fiersmi01", "ohse01", "lemahdj01","fiersmi01", "wheelza01", "claudal01", "joycema01", "edwarca01", "riddljt01", "winklda01"]

In [19]:
len(NaNID) == len(Name_NaNID)

True

Through the process, I accidentally figured out row 523 Jeremy McBryde have not made it to the Majors yet, so will leave this row out of the analysis for now. There might be other players that have not played in the Majors but appears in the dataset. 

## Players with multiple bbrefID

In [20]:
# The dataframe of players with multiple bbrefID
FA_list_duplicated = FA_list_joined[FA_list_joined.drop(["bbrefID", "finalGame","birthYear","_merge"], axis = 1).duplicated(keep = False)]
FA_list_duplicated

,Name,Position,Age,From,To,Years,Dollars,Average_Salary,FA_year,bbrefID,finalGame,birthYear,_merge
2,Jose Reyes,SS,29,NYM,MIA,6,"$106,000,000","$17,666,667",2012,reyesjo01,2018-09-30,1983.0,both
3,Jose Reyes,SS,29,NYM,MIA,6,"$106,000,000","$17,666,667",2012,reyesjo02,2006-09-30,1983.0,both
32,Alex Gonzalez,SS,35,ATL,MIL,1,"$4,250,000","$4,250,000",2012,gonzaal01,2006-05-20,1973.0,both
33,Alex Gonzalez,SS,35,ATL,MIL,1,"$4,250,000","$4,250,000",2012,gonzaal02,2014-04-17,1977.0,both
34,Freddy Garcia,SP,35,NYY,NYY,1,"$4,000,000","$4,000,000",2012,garcifr01,1999-10-03,1972.0,both
35,Freddy Garcia,SP,35,NYY,NYY,1,"$4,000,000","$4,000,000",2012,garcifr03,2013-09-24,1976.0,both
80,Chris Young,RP,33,NYM,NYM,1,"$900,000","$900,000",2012,youngch03,2017-06-17,1979.0,both
81,Chris Young,RP,33,NYM,NYM,1,"$900,000","$900,000",2012,youngch04,2018-07-03,1983.0,both
159,Scott Baker,SP,31,MIN,CHC,1,"$5,500,000","$5,500,000",2013,bakersc01,1995-07-17,1970.0,both
160,Scott Baker,SP,31,MIN,CHC,1,"$5,500,000","$5,500,000",2013,bakersc02,2015-05-02,1981.0,both


There are 2 ways to help determine which bbrefID to keep (which row to keep):   <br/> 
1) Check finalGame: If the player played his final game before the FA contract, it would not make sense.   <br/>
2) Check age using birthYear: The age of signing should be the same as the year of signing minus birthYear.

In [21]:
# Make finalGame a year integer 
finalGame = list()
for i in FA_list_duplicated["finalGame"].index:
    finalGame.append(FA_list_duplicated["finalGame"][i][0:4])
finalGame_int = list()
for i in range(len(finalGame)):
    finalGame_int.append(int(finalGame[i]))
FA_list_duplicated.iloc[:,10] = finalGame_int

C:\Users\iamia\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [22]:
# Make birthYear and finalGame integer
birthYear_int = FA_list_duplicated["birthYear"].astype(int)
FA_list_duplicated.iloc[:,11] = birthYear_int
# Have to make an object and put it back

In [23]:
keep_finalGame = list()
age_difference = list()
for i in range(len(FA_list_duplicated)):
    keep_finalGame.append(FA_list_duplicated.iloc[i,10] > 2010)  # True if finalGame is later than 2010
    age_difference.append(FA_list_duplicated.iloc[i,8] - FA_list_duplicated.iloc[i,11] - FA_list_duplicated.iloc[i,2])

In [24]:
# Allow +- 1 difference in age and FA_year - birthDate
keep_Age = list()
for i in range(len(age_difference)):
    if age_difference[i] in [-1, 0, 1]:
        keep_Age.append(True)
    else: 
        keep_Age.append(False)

In [25]:
FA_list_duplicated["Keep_Age"] = keep_Age
FA_list_duplicated["Keep_finalGame"] = keep_finalGame

C:\Users\iamia\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\iamia\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [25]:
FA_list_duplicated

,Name,Position,Age,From,To,Years,Dollars,Average_Salary,FA_year,bbrefID,finalGame,birthYear,_merge,Keep_Age,Keep_finalGame
2,Jose Reyes,SS,29,NYM,MIA,6,"$106,000,000","$17,666,667",2012,reyesjo01,2018,1983,both,True,True
3,Jose Reyes,SS,29,NYM,MIA,6,"$106,000,000","$17,666,667",2012,reyesjo02,2006,1983,both,True,False
32,Alex Gonzalez,SS,35,ATL,MIL,1,"$4,250,000","$4,250,000",2012,gonzaal01,2006,1973,both,False,False
33,Alex Gonzalez,SS,35,ATL,MIL,1,"$4,250,000","$4,250,000",2012,gonzaal02,2014,1977,both,True,True
34,Freddy Garcia,SP,35,NYY,NYY,1,"$4,000,000","$4,000,000",2012,garcifr01,1999,1972,both,False,False
35,Freddy Garcia,SP,35,NYY,NYY,1,"$4,000,000","$4,000,000",2012,garcifr03,2013,1976,both,True,True
80,Chris Young,RP,33,NYM,NYM,1,"$900,000","$900,000",2012,youngch03,2017,1979,both,True,True
81,Chris Young,RP,33,NYM,NYM,1,"$900,000","$900,000",2012,youngch04,2018,1983,both,False,True
159,Scott Baker,SP,31,MIN,CHC,1,"$5,500,000","$5,500,000",2013,bakersc01,1995,1970,both,False,False
160,Scott Baker,SP,31,MIN,CHC,1,"$5,500,000","$5,500,000",2013,bakersc02,2015,1981,both,True,True


In [26]:
print(FA_list_duplicated.shape)
print(FA_list_duplicated[FA_list_duplicated["Keep_Age"] & FA_list_duplicated["Keep_finalGame"]].shape)

(56, 15)
(30, 15)


In [27]:
# Keep those with two trues
FA_list_duplicated[FA_list_duplicated["Keep_Age"] & FA_list_duplicated["Keep_finalGame"]]

,Name,Position,Age,From,To,Years,Dollars,Average_Salary,FA_year,bbrefID,finalGame,birthYear,_merge,Keep_Age,Keep_finalGame
2,Jose Reyes,SS,29,NYM,MIA,6,"$106,000,000","$17,666,667",2012,reyesjo01,2018,1983,both,True,True
33,Alex Gonzalez,SS,35,ATL,MIL,1,"$4,250,000","$4,250,000",2012,gonzaal02,2014,1977,both,True,True
35,Freddy Garcia,SP,35,NYY,NYY,1,"$4,000,000","$4,000,000",2012,garcifr03,2013,1976,both,True,True
80,Chris Young,RP,33,NYM,NYM,1,"$900,000","$900,000",2012,youngch03,2017,1979,both,True,True
160,Scott Baker,SP,31,MIN,CHC,1,"$5,500,000","$5,500,000",2013,bakersc02,2015,1981,both,True,True
187,Chris Young,RP,34,NYM,WSH,1,"$2,000,000","$2,000,000",2013,youngch03,2017,1979,both,True,True
205,Alex Gonzalez,SS,36,MIL,MIL,1,"$1,450,000","$1,450,000",2013,gonzaal02,2014,1977,both,True,True
295,Nelson Cruz,DH,33,TEX,BAL,1,"$8,000,000","$8,000,000",2014,cruzne02,2019,1980,both,True,True
301,Chris Young,RF,30,OAK,NYM,1,"$7,250,000","$7,250,000",2014,youngch04,2018,1983,both,True,True
323,Francisco Rodriguez,RP,32,BAL,MIL,1,"$3,250,000","$3,250,000",2014,rodrifr03,2017,1982,both,True,True


There are only 3 pairs of duplicates that we have to handle manually: Francisco Rodriguez(x2) and Miguel Gonzalez.  <br/>
By manually checking, bbrefID for Francisco Rodriguez is rodrifr03 and bbrefID for Miguel Gonzalez is gonzami03.   <br/>
This means index 324, 435, and 852 are false. 

In [28]:
FA_list_duplicated.shape

(56, 15)

In [29]:
# The index of rows we should keep
Keep_Index = FA_list_duplicated[FA_list_duplicated["Keep_Age"] & FA_list_duplicated["Keep_finalGame"]].index

In [30]:
len(Keep_Index)

30

In [31]:
# Drop the 3 manually.
Keep_Index = Keep_Index.drop([324, 435, 852])

In [32]:
NA_Index = list()
for i in FA_list_duplicated.index:
    NA_Index.append(i not in Keep_Index)
# Returns True if it should be dropped.
# A list of True/False

In [33]:
FA_list_duplicated.index[NA_Index] # The index that should be dropped from FA_list

Int64Index([   3,   32,   34,   81,  159,  188,  204,  294,  300,  324,  355,
             372,  413,  435,  492,  533,  569,  575,  619,  690,  754,  851,
             852,  871,  875,  927, 1111, 1112, 1120],
           dtype='int64')

In [34]:
len(FA_list_duplicated.index[NA_Index])

29

## Put back together 

In [35]:
len(bbrefID_NA_index)

33

In [36]:
# Fill in the NaNs in bbrefID
for i in range(len(NaNID)):
    FA_list_joined.at[bbrefID_NA_index[i], 'bbrefID'] = NaNID[i]

In [37]:
FA_list_joined = FA_list_joined.drop(FA_list_duplicated.index[NA_Index])
# Now FA_list_joined has the same number of rows as FA_list

In [38]:
FA_list_joined.shape

(1118, 13)

In [39]:
# Remove last three column 
FA_list_joined.drop(["finalGame", "birthYear", "_merge"], axis = 1, inplace = True)
FA_list_joined

,Name,Position,Age,From,To,Years,Dollars,Average_Salary,FA_year,bbrefID
0,Albert Pujols,DH,32,STL,LAA,10,"$240,000,000","$24,000,000",2012,pujolal01
1,Prince Fielder,DH,28,MIL,DET,9,"$214,000,000","$23,777,778",2012,fieldpr01
2,Jose Reyes,SS,29,NYM,MIA,6,"$106,000,000","$17,666,667",2012,reyesjo01
4,C.J. Wilson,SP,31,TEX,LAA,5,"$77,500,000","$15,500,000",2012,wilsocj01
5,Mark Buehrle,SP,33,CHW,MIA,4,"$58,000,000","$14,500,000",2012,buehrma01
...,...,...,...,...,...,...,...,...,...,...
1142,Dustin Garneau,C,32,OAK,HOU,1,"$650,000","$650,000",2020,garnedu01
1143,Collin McHugh,SP,33,HOU,BOS,1,"$600,000","$600,000",2020,mchugco01
1144,Jesse Hahn,RP,30,KC,KC,1,"$600,000","$600,000",2020,hahnje01
1145,Luke Maile,C,29,TOR,PIT,1,"$583,500","$583,500",2020,mailelu01


In [40]:
# Take away the dollar sign and comma in Dollars and Average_Salary
FA_list_joined["Dollars"] = FA_list_joined["Dollars"].str.replace("$", "")
FA_list_joined["Dollars"] = FA_list_joined["Dollars"].str.replace(",", "")
FA_list_joined["Average_Salary"] = FA_list_joined["Average_Salary"].str.replace("$", "")
FA_list_joined["Average_Salary"] = FA_list_joined["Average_Salary"].str.replace(",", "")

In [26]:
FA_list_joined.head()

,Name,Position,Age,From,To,Years,Dollars,Average_Salary,FA_year,bbrefID,finalGame,birthYear,_merge
0,Albert Pujols,DH,32,STL,LAA,10,"$240,000,000","$24,000,000",2012,pujolal01,2019-09-29,1980.0,both
1,Prince Fielder,DH,28,MIL,DET,9,"$214,000,000","$23,777,778",2012,fieldpr01,2016-07-18,1984.0,both
2,Jose Reyes,SS,29,NYM,MIA,6,"$106,000,000","$17,666,667",2012,reyesjo01,2018-09-30,1983.0,both
3,Jose Reyes,SS,29,NYM,MIA,6,"$106,000,000","$17,666,667",2012,reyesjo02,2006-09-30,1983.0,both
4,C.J. Wilson,SP,31,TEX,LAA,5,"$77,500,000","$15,500,000",2012,wilsocj01,2015-07-28,1980.0,both


2 changes are required during the analysis 
1) Tyler Colvin (colvity01) should be an outfielder instead of a RP
2) Get rid of Tony Barnettes 2016 FA becuase he has not pitched in the Majors before 2016

In [46]:
FA_list_joined.iloc[353,1] = "OF"
FA_list_joined = FA_list_joined.drop(611)

In [47]:
# Output data
FA_list_joined.to_csv("FA_list_with_bbrefID.csv", index = False)